In [ ]:
import multiprocessing
import sys
sys.path.insert(0,'python')
from fastRWpkl import *
import numpy.ma as ma
from readSent import *
from collections import Counter
import cPickle as pkl
from scipy import optimize
from functools import partial
import scipy.ndimage as ndimage
import numpy as np
import scipy
from scipy import signal
import scipy.stats

fhead = 'data/50SMG20165100'

def gaussian(xwin, ywin, xstd, ystd, angle, norm = True):
    win = max(xwin, ywin)
    winx = win*2**0.5
    winy = win*2**0.5
    
    xstd = xstd*2**0.5
    ystd = ystd*2**0.5
        
    xgaus = signal.gaussian(winx, xstd)
    ygaus = signal.gaussian(winy, ystd)
    gaus  = np.outer(xgaus, ygaus)
    r_gaus = scipy.ndimage.interpolation.rotate(gaus, angle, reshape=True)
    center = np.array(r_gaus.shape)/2
    cgaus = r_gaus[center[0]-xwin/2: center[0]+xwin/2, center[1]-ywin/2:center[1]+ywin/2]
    if norm:
        return cgaus/cgaus.sum()
    else:
        return cgaus

def shift(process):
    store = []
    for i, j in process:
        print i,j

        s1 = slice((i*1000),(i+1)*1000)
        s2 = slice((j*1000),(j+1)*1000)
        modis_cut = modis_sent[s1,s2]*0.001
        Stm_cut = Stm[s1,s2]

        sen_cut = Sent[s1,s2]
        ulist = np.unique(Stm_cut)
        
        
        for ii,u in enumerate(ulist):
            mask = (Stm_cut == u)
            if (mask.sum() >= 1800) and (mask.sum() < 4000):
                xmin = np.where(mask)[0].min()
                xmax = np.where(mask)[0].max()
                ymin = np.where(mask)[1].min()
                ymax = np.where(mask)[1].max()
                center0 = np.array([(xmax+xmin)/2, (ymax+ymin)/2])

                minx = center0[0] - 100
                maxx = center0[0] + 100
                miny = center0[1] - 100
                maxy = center0[1] + 100

                offset = -0.077837880645250962
                to_conv = sen_cut[max(0, minx): min(999, maxx), max(0, miny): min(999, maxy)]
                brdf = modis_cut[center0[0],center0[1]]
                
                
                if (to_conv.shape[0]>100) & (to_conv.shape[1]>100) & (brdf!=np.nan):
                    To_conv = to_conv + offset
                    Brdf = brdf
                    nanval = np.where(~((To_conv < 1)&(To_conv > 0)))
                    To_conv[nanval[0], nanval[1]] = np.nanmean(To_conv)
                    p = np.array([41.8725426787, 24.7544561511, 46.6626096076])
                    bound = np.array([(8.,200.),(8.,205.),(20.,187.)])
                    psolve = optimize.fmin_l_bfgs_b(cost,p,approx_grad=True,iprint=-1,\
                                                    args=(To_conv, Brdf),bounds=bound)

                    xstd, ystd, angle = psolve[0]
                    xwin,ywin = 100,100
                    gaus = gaussian(xwin,ywin,xstd,ystd,angle,False)
                    if gaus.sum() <= 0:
                        print 'invalid gauss: %s'%([xstd, ystd, angle])
                        pass
                    else:
                        kernel = gaus/(gaus.sum())    
                        s = signal.fftconvolve(To_conv, kernel, mode='valid')
                        dif = abs(s-brdf)
                        mindif = np.nanmin(dif)
                        # find out the shift in x and y direction, with the minimum distance accepted
                        cx,cy = np.array(s.shape)/2
                        sx,sy = np.where(dif == mindif)
                        shift = zip(sx-cx,sy-cy)
                        minsft = sorted(shift, key=lambda x: abs(x[1])+abs(x[0]))[0]

                        conved = to_conv[cx+minsft[0], cy+minsft[1]]
                        store.append([u, xstd, ystd, angle, conved, brdf, minsft[0], minsft[1]])

                        print 'solved one: ', ii, u, xstd, ystd, angle, conved, brdf, minsft
                else:
                    pass

            else:
                pass
    return store

def cost(p, To_conv = None, Brdf = None):

    xstd,ystd,angle = p
    #print p, (~np.isnan(To_conv)).all()

    xwin,ywin = 100,100
    gaus = gaussian(xwin,ywin,xstd,ystd,angle,False)                            

    
    ker = gaus/(gaus.sum())    
    s = signal.fftconvolve(To_conv, ker, mode='valid')
    dif = abs(s-Brdf)
    mindif = np.nanmin(dif)
    #print mindif
    return mindif



def ScaleExtent(data, shape): # used for unifine different array,

    re = int(shape[0]/(data.shape[0]))

    a = np.repeat(np.repeat(data, re, axis = 1), re, axis =0)
    
    if (re*(data.shape[0])-shape[0]) != 0:
        extended = np.zeros(shape)
        extended[:re*(data.shape[0]),:re*(data.shape[0])] = a
        extended[re*(data.shape[0]):,re*(data.shape[0]):] = a[re*(data.shape[0])-shape[0]:, re*(data.shape[0])-shape[0]]
        return extended
    else:
        return a

ind = pkl.load(open('pkls/gaus_trans.pkl','rb'))
inds = ind
data = parallel_rw_pkl(None, 'inter_sent%i'%0, 'r')
mask = parallel_rw_pkl(None, 'inter_sentm%i'%0, 'r')
Sent = readfile([2,],fhead)['B02']
cm = parallel_rw_pkl(None, '0510diacm', 'r')
Sent[cm] = np.nan
stm = parallel_rw_pkl(None, 'std_m', 'r')
print 'finshed reading data'
data[mask]=np.nan
modis_sent = np.array(data)
Stm = stm
patches = np.array(zip(np.mgrid[0:10,0:10][0].ravel(), np.mgrid[0:10,0:10][1].ravel()))
pros = np.array(np.array_split(patches, 16))

par = partial(shift)
pool = multiprocessing.Pool(processes=16)
data = pool.map(par, pros)
pool.close()
pool.join()
parallel_rw_pkl(data, 'psfb2_modis', 'w')

print 'lol finishedb2!!!!!'

In [ ]:
import gdal
import numpy as np
import numpy.ma as ma
import sys
sys.path.insert(0,'python')
from readSent import *
import kernels
from geo_trans import *
from scipy.interpolate import griddata
from fast_rw import *
import matplotlib 
matplotlib.use('Agg')

import pylab as plt


modis_filenames = gdal.Open('m_data/MCD43A1.A2016130.h27v05.006.2016180225447.hdf').GetSubDatasets()
modisQA = gdal.Open("m_data/MCD43A2.A2016131.h27v05.006.2016180231424.hdf").GetSubDatasets()
h=27
v=5
fhead = 'data/50SMG20165100'

def get_coords(h,v):
    mgrss = get_lon_lat(27, 5).ravel()

    mgrss = np.array([(i[:5],i[-8:-4],i[-4:]) for i in mgrss]).reshape(2400,2400,3)

    index = np.where(mgrss[:,:,0]=='50SMG')
    d = mgrss[index[0],index[1],:]
    Scoords = [9999-d[:,2].astype('int'), d[:,1].astype('int')]
    return index, Scoords

def r_modis(fname):
    g = gdal.Open(fname)
    if g is None:
        raise IOError
    else:
        return g.ReadAsArray()


def ScaleExtent(data, shape): # used for unifine different array,

    re = int(shape[0]/(data.shape[0]))

    a = np.repeat(np.repeat(data, re, axis = 1), re, axis =0)
    
    if re*(data.shape[0]-shape[0]) != 0:
        extended = np.zeros(shape)
        extended[:re*(data.shape[0]),:re*(data.shape[0])] = a
        extended[re*(data.shape[0]):,re*(data.shape[0]):] = a[re*(data.shape[0])-shape[0]:, re*(data.shape[0])-shape[0]]
        return extended
    else:
        return a
#bands = [2,3,4,8,13,11,12]


def get_kk(fhead):
    mete = readxml('%smetadata.xml'%fhead)
    vZa = np.array([i+ np.zeros((23,23)) for i in mete['mVz']])
    vAa = np.array([i+ np.zeros((23,23)) for i in mete['mVa']])
    sza = mete['mSz'][0]
    vza = mete['mVz'][3]
    rel_a = (mete['mSa']-mete['mVa'])[3]
    kk = kernels.Kernels(vza ,sza,rel_a,\
                         RossHS=False,MODISSPARSE=True,\
                         RecipFlag=True,normalise=1,\
                         doIntegrals=False,LiType='Dense',RossType='Thick')
    return kk



def get_rs(modisQA, modis_filenames, fhead):
    
    kk = get_kk(fhead)
    k_vol = kk.Ross
    k_geo   = kk.Li
    iso =  kk.Isotropic
    
    brdfs = []
    #QA = r_modis(modisQA[0][0])


    for i in [2,3,0,1,5,6]:
        br = r_modis(modis_filenames[i+10][0])
        QA = r_modis(modis_filenames[i][0])
        
        mask = (br[0] > 32766) | (br[1] > 32766) |(br[2] > 32766)| (QA>1)
        brdf = br[0] + br[1]*k_vol + br[2]*k_geo
        brdf = ma.array(brdf, mask = mask)
        brdfs.append(brdf)
        
    return brdfs

def inter_modis(modisQA, modis_filenames, fhead, h, v):
    
    brdfs = get_rs(modisQA, modis_filenames, fhead)
    print brdfs[0].shape

    grid_x, grid_y = np.mgrid[0:10980, 0:10980]
    index, Scoords = get_coords(h,v)
    print index, Scoords
    for i in xrange(6):
        sentm = np.zeros((10980,10980))
        brd = brdfs[i]     
        values = brd[index[0],index[1]]  
        inter_sent = griddata(np.array(zip(Scoords[0],Scoords[1])), values, (grid_x, grid_y), method='nearest')
        parallel_rw_pkl(inter_sent.data, 'C6_modis%i'%i, 'w')
        parallel_rw_pkl(inter_sent.mask, 'C6_modism%i'%i, 'w')
inter_modis(modisQA, modis_filenames, fhead, h, v)


In [29]:
modis_filenames = gdal.Open('m_data/MCD43A2.A2016131.h27v05.006.2016180231424.hdf').GetSubDatasets()